# Data Scientist (DS) - Alice's Notebook

This notebook demonstrates the **RemoteData architecture** from the data scientist's perspective.

Alice wants to analyze patient data owned by Bob, but Bob wants to preserve privacy.

**⚠️  Make sure you've run `test_do.ipynb` first!**

## Setup

In [1]:
import sys
sys.path.insert(0, "../python/src")

import beaver
import pandas as pd
from beaver import Twin

In [2]:
# Connect as Alice (data scientist)
bv = beaver.connect("shared", user="alice")

🔄 Auto-load replies enabled for alice (polling every 2.0s)


## 1. Check Inbox

Alice received data from Bob. Let's see what's in the inbox.

In [3]:
bv.peer("bob").remote_vars

Name,Type,ID
counter,Twin[int],8abe8c7c1ff1...


In [4]:
bv.peer("bob").remote_vars["counter"].load()

🟢 Live sync enabled (read-only, every 2.0s)
✓ Loaded Twin 'counter' from published location


🌍 Twin: counter (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public    7    ← .value uses this
  Owner: bob
  Live: 🟢 Enabled (read-only, 2.0s)
  💡 Owner's Twin is live-syncing
  IDs: twin=6bbb4062... private=71fab274... public=7f9f9624...

In [5]:
counter

🌍 Twin: counter (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public    7    ← .value uses this
  Owner: bob
  Live: 🟢 Enabled (read-only, 2.0s)
  💡 Owner's Twin is live-syncing
  IDs: twin=6bbb4062... private=71fab274... public=7f9f9624...

In [6]:
counter.public

7

In [7]:
counter.public

7

In [8]:
counter.public_value

7

In [10]:
for i in range(100):
    import time
    time.sleep(2)
    print(counter.public)

KeyboardInterrupt: 

In [11]:
bv.workspace()

Name,Owner,Type,Status,Last Action
counter,bob,Twin[int],🟢 Live,31s ago
patient_data,bob,Twin[Twin],📥 Received,32s ago


In [12]:
# View inbox
bv.inbox()

name,id,sender,type,size_bytes,created_at,reply_to
patient_data,a03df99a7904...,bob,Twin,1232,2025-11-20 06:56:14,


In [13]:
# Look at the first envelope
bv.inbox()[0]

BeaverEnvelope [DATA]: patient_data
  From: bob
  Type: Twin (beaver.twin)
  Size: 1232 bytes
  Created: 2025-11-20 06:56:14 UTC
  ID: a03df99a...

Preview:
  ⚠️  Twin: patient_data (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private      patient_id           name  age  test_result diagnosis
0...    ← .value uses this
  🌍 Public      patient_id       name  age  test_result diagnosis
0    ...    ✓
  Live: ⚫ Disabled
  IDs: twin=1671f0e6... private=f0f04ee1... public=1701d52e...

## 2. Load the Twin (Public Side Only)

When Alice loads the Twin, she **only receives the public (mock) data**.
The private data never left Bob's environment!

In [14]:
# Load the Twin from Bob
bv.inbox()[-1].load()

✓ Loaded 'patient_data' into globals


🌍 Twin: patient_data (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public      patient_id       name  age  test_result diagnosis
0    ...    ← .value uses this
  Owner: bob
  Live: ⚫ Disabled
  IDs: twin=1671f0e6... private=f0f04ee1... public=1701d52e...

In [15]:
# Now patient_data is in globals
patient_data

🌍 Twin: patient_data (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public      patient_id       name  age  test_result diagnosis
0    ...    ← .value uses this
  Owner: bob
  Live: ⚫ Disabled
  IDs: twin=1671f0e6... private=f0f04ee1... public=1701d52e...

In [16]:
# Check what Alice has access to
print(f"Has private: {patient_data.has_private}")  # False!
print(f"Has public: {patient_data.has_public}")    # True!
print(f"Owner: {patient_data.owner}")

Has private: False
Has public: True
Owner: bob


## 3. Work with Public (Mock) Data

Alice can develop her analysis using the mock data.

In [17]:
# Alice works with mock data
print("🌍 Working with PUBLIC data:")
patient_data.value

🌍 Working with PUBLIC data:
🌍 Using PUBLIC data from Twin 'patient_data...'


,patient_id,name,age,test_result,diagnosis
0,M001,Patient A,30,6.5,negative
1,M002,Patient B,40,8.0,positive
2,M003,Patient C,35,7.0,positive
3,M004,Patient D,50,9.0,positive
4,M005,Patient E,45,8.5,positive


In [18]:
# Develop analysis on mock data
print(f"Total patients (mock): {len(patient_data.value)}")
print(f"Average age (mock): {patient_data.value['age'].mean():.1f}")
print(f"Positive rate (mock): {(patient_data.value['diagnosis'] == 'positive').sum() / len(patient_data.value) * 100:.1f}%")

Total patients (mock): 5
Average age (mock): 40.0
Positive rate (mock): 80.0%


In [19]:
# Create analysis function
@bv
def analyze_patients(df):
    """Analysis function developed on mock data."""
    results = {
        'total_patients': len(df),
        'avg_age': df['age'].mean(),
        'avg_test_result': df['test_result'].mean(),
        'positive_count': (df['diagnosis'] == 'positive').sum(),
        'positive_rate': (df['diagnosis'] == 'positive').sum() / len(df) * 100
    }
    return results

In [20]:
# Test on mock data
mock_results = analyze_patients(patient_data.public)
print("\n📊 Analysis Results (Mock Data):")
for key, value in mock_results.items():
    print(f"  {key}: {value:.2f}")


📊 Analysis Results (Mock Data):
  total_patients: 5.00
  avg_age: 40.00
  avg_test_result: 7.80
  positive_count: 4.00
  positive_rate: 80.00


In [21]:
type(mock_results)

dict

## 4. Request Private Data Access

Once Alice's analysis is ready, she can request access to the **real (private)** data.

In [22]:
real_result = analyze_patients(patient_data)

In [23]:
real_result

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=ecee67ab... private=0906cab2... public=88193eda...

In [24]:
real_result.request_private()

📨 Sending computation request to bob
   Function: analyze_patients
   Result: real_result
✓ Sent to shared/bob/45e80be313024c57b05075ff87a30a5b.beaver
💡 Result will auto-update when bob approves


In [25]:
real_result

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=ecee67ab... private=0906cab2... public=88193eda...

In [38]:
bv.workspace()

Name,Owner,Type,Status,Last Action
analyze_patients_result,bob,dict,📥 Received,6m ago
counter,bob,Twin[int],🟢 Live,7m ago
patient_data,bob,Twin[Twin],📥 Received,7m ago


In [26]:
real_result

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=ecee67ab... private=0906cab2... public=88193eda...

✨ Twin result auto-updated: analyze_patients_result...
   .value now uses private (approved result)


In [27]:
real_result.value

🔒 Using PRIVATE data from Twin 'analyze_patients_result...'


{'total_patients': 10,
 'avg_age': np.float64(40.9),
 'avg_test_result': np.float64(7.92),
 'positive_count': np.int64(8),
 'positive_rate': np.float64(80.0)}

In [28]:
real_result

⚠️  Twin: analyze_patients_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    {'total_patients': 10, 'avg_age': np.float64(40.9), 'avg_...    ← .value uses this
  🌍 Public    {'total_patients': 5, 'avg_age': np.float64(40.0), 'avg_t...    ✓
  Live: ⚫ Disabled
  IDs: twin=ecee67ab... private=0906cab2... public=88193eda...

In [30]:
bv.inbox()

name,id,sender,type,size_bytes,created_at,reply_to
patient_data,a03df99a7904...,bob,Twin,1232,2025-11-20 06:56:14,
analyze_patients_result,8dff4aaad8f2...,bob,dict,321,2025-11-20 06:57:03,358311c0739d...


In [31]:
# Request private access
patient_data.request_private()

🔒 Requesting private data access from bob
   Twin: patient_data
   💡 (Data access request flow not yet fully implemented)


💡 **Note**: The request flow is not yet fully implemented. In a complete system:
- Bob would receive Alice's request
- Bob could review the analysis code
- Bob could approve and send results (computed on private data)
- Or Bob could grant temporary access with conditions

## 5. Access Remote Variables

Alice can see what remote variables Bob has published.

In [32]:
# View Bob's remote variables
bv.peer("bob").remote_vars

Name,Type,ID
counter,Twin[int],8abe8c7c1ff1...


In [33]:
bv.workspace()

Name,Owner,Type,Status,Last Action
analyze_patients_result,bob,dict,📥 Received,25s ago
counter,bob,Twin[int],🟢 Live,1m ago
patient_data,bob,Twin[Twin],📥 Received,1m ago


In [ ]:
bv.peer("bob").remote_vars["patient_data"].load()

In [ ]:
patient_data.live